In [5]:

import numpy as np
import pandas as pd
import os
import pyodbc
from dotenv import load_dotenv
from dotenv import dotenv_values
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Loading the 2018 dataset
df_2018N = pd.read_csv('./Dataset/startup_funding2018.csv') 

#Loading the 2019 dataset
df_2019N = pd.read_csv('./Dataset/startup_funding2019.csv') 

#Loading 2020 and 2021 datasets
# Load environment variables from .env file
environment_variables = dotenv_values('.env')

# Access database credentials from environment variables dictionary
server = environment_variables.get("SERVER")
database = environment_variables.get("DATABASE")
password = environment_variables.get("PASSWORD")
username = environment_variables.get("USERNAME")

# Construct the connection string
connection_string = f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};User Id={username};PASSWORD={password};"

print("USERNAME:", username)
print("PASSWORD:", password)

# Construct the connection string
connection_string = f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password};"
# Connect to the database
try:
    connection = pyodbc.connect(connection_string)
    print("Connection successful!")
except Exception as e:
    print("Error:", e)

# Specify the SQL queries to extract data from the tables
query_2020N = "SELECT * FROM dbo.LP1_startup_funding2020"
query_2021N = "SELECT * FROM dbo.LP1_startup_funding2021"

# Suppress warnings
warnings.filterwarnings('ignore')

# Create a cursor from the connection
with connection.cursor() as cursor:
    # Execute the queries and fetch data into Pandas DataFrames
    df_2020N = pd.read_sql_query(query_2020N, connection)
    df_2021N = pd.read_sql_query(query_2021N, connection)
